In [ ]:
#importing req library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image

#Importing libraries for defining the architechure
import torch
import torch.nn as nn
from torch.optim import adam
from torch.nn import ReLU,BCELoss,Sequential,Sigmoid,Linear

#importing torch vision
from torchvision.models import googlenet
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))  # 0 corresponds to the first GPU

##  Loading and pre-processing the Dataset

In [ ]:
csv_path="G:\\NN\\Assment\\Emergency and non emergency vechile clasification\\Dataset\\emergency_classification.csv"
img_dir='G:/NN/Assment/Emergency and non emergency vechile clasification/Dataset/images/'

In [ ]:
#defining the preprocessing steps
normalize=transforms.Normalize(mean=[0.485,0.456,0.406],
                               std=[0.229,0.224,0.225])
preprocessing=transforms.Compose([transforms.ToTensor(),normalize])

In [ ]:
# defining the class to load dataset 
class EmergencyDataset(Dataset):
    """Custom Dataset for loading Emergency Dataset"""

    # defining the init function
    def __init__(self, csv_path, img_dir, transform):
        df = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.csv_path = csv_path
        self.img_names = df.image_names.values
        self.y = df['emergency_or_not'].values
        self.transform = transform

    # defining the get item function
    def __getitem__(self, index):
        img = Image.open(self.img_dir + self.img_names[index])

        if self.transform is not None:
            img = self.transform(img)

        label = self.y[index]
        return img, label

    # defining the len function
    def __len__(self):
        return self.y.shape[0]


In [ ]:
# loading the dataset
train_dataset = EmergencyDataset(csv_path,img_dir,
                              transform=preprocessing)

In [ ]:
#loading the data in batch using dataloader
trainloader=DataLoader(dataset=train_dataset,
                       batch_size=45,
                       shuffle=True)


In [ ]:
for batch_idx ,(batch_x,batch_y)in enumerate(trainloader):
    break

In [ ]:
#shape of the label
batch_x.shape,batch_y.shape

##  loading the weights of the pre trained model

In [ ]:
# define model architecture along with pretrained weights of googlenet / inception_v1
googlenet_model=googlenet(pretrained=True)

In [ ]:
#print the model
googlenet_model

In [ ]:
# architecture in form of a list
list(googlenet_model.children())

In [ ]:
#defining the extract model
class FreatureExtractor(nn.Module):
    def __init__(self):
        super(FreatureExtractor,self).__init__()
        #push to cuda
        if torch.cuda.is_available():
            self.net=self.net.cuda()
        for p in self.net.parameters():
            p.requires_grad=False
            # Define which layers you are going to extract
        self.feauters=nn.Sequential(*list(self.net.childern())[:-3])
    def froward(self,x):
        return self.feauters(x)


In [ ]:
# prepare input
input = batch_x[:2]
input = input.cuda()

# pass the input to vgg16
if __name__ == "__main__":
    fe = FeatureExtractor()
    output = fe(input)

# shape of the output
output.shape


In [ ]:
!conda install pytorch torchvision torchaudio cudatoolkit=12.3 -c pytorch

 conda install conda=24.1.0

In [ ]:

#shape of output
output.shape

In [ ]:
torch.cuda.is_available()